In [1]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
import random
import io
import string
from sklearn.metrics.pairwise import cosine_similarity
import MeCab 
import numpy as np

In [2]:
#p = Path("/home/ifte/amiebot_project/amie-HelpBot/mobicontrol_data/corpus_mobicontrol.csv")
df = pd.read_csv("/home/ifte/amiebot_project/Large_files/mobicontrol_data/Train_Test_Data/japanese/corpus_ver14.csv", 
                 header=0, usecols=["page", "text"])

In [122]:
df[df['text'].str.contains('iMessage')]

,page,text,parsed
408,1,iMessageの利用禁止,利用 禁止
46532,271,iMessageの利用禁止,利用 禁止
46731,271,iMessageの利用禁止,利用 禁止
68698,430,iMessageの利用禁止,利用 禁止


In [4]:
stop_words_ja = []
STOPWORD_FILE = Path("/home/ifte/amiebot_project/amie-HelpBot/amie_helpbot/" + '/assets/learning/stop_word.txt')
with open(STOPWORD_FILE, encoding='utf-8') as fr:
    stop_words_ja = fr.read().splitlines()

In [5]:
import re

def clean_text(text):
    replaced_text = re.sub(r'[【】]', ' ', text)       # 【】の除去
    replaced_text = re.sub(r'[・_]', '', replaced_text)       # ・ の除去
    replaced_text = re.sub(r'[（）()]', ' ', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', ' ', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[@＠]\w+', '', replaced_text)  # メンションの除去
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', ' ', replaced_text)  # 全角空白の除去
    replaced_text = re.sub(r'\d+', '', replaced_text) # 数字の除去
    replaced_text = re.sub(r'[-/。,、.=]', ' ', replaced_text)
    
    return replaced_text

In [6]:
# Developed Tagger Parser
	def cleaning(text, stop_words):
		# 形態素解析
		tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
		tagger.parse("")
		res = tagger.parseToNode(text.lower())
		str_out = ''
		
		while res:
			arr = res.feature.split(",")
		
			# 辞書に登録されている名詞のみ
			if not arr == None and len(arr) > 7 and arr[0] == '名詞':
				word = arr[6].lower()
				
				# 数値、ストップワード、アルファベット1文字は除外
				if not arr[1] == '数' and not word in stop_words and \
					not (len(word) == 1 and re.search(r"[a-xA-Z0-9]", word) is not None):
					str_out += word + ' '
			res = res.next
		
		return str_out


In [7]:
y = df[["page"]]
X = df["text"].apply(lambda x: cleaning(x,stop_words_ja).strip("\n").rstrip())
df['parsed'] = X 

classes = df.page.unique()

In [8]:
            if set(['page', 'text']).issubset(df.columns):
                pages = df.page.unique()
                temp_dict = {}
                for i in list(pages):
                    page_content = df[df.page == i].text.values
                    data = ''.join(list(page_content))
                    temp_dict[i] = [data]
                processed_df = pd.DataFrame.from_dict(temp_dict, orient='index', columns=['Data'])
                processed_df['PageID'] = processed_df.index

In [10]:
processed_df.head()

,Data,PageID
0,カスタマイザーからこのタイトルを設定できます。カスタマイザーからこのサブタイトルを設定できま...,0
1,Apple製品MobiControl v14 ManualApple製品をMobiContr...,1
2,Android端末MobiControl v14 ManualAndroidの設定モードA....,2
3,Windows PCMobiControl v14 ManualWindows PCの根本的...,3
4,Windows EmbeddedMobiControl v14 ManualWindows ...,4


In [11]:
processed_df.to_csv("processed.csv", index=False)

In [9]:
                    for index, row in processed_df.iterrows():
                        #print(row['Data'])
                        test=row['Data']
                        break

In [10]:
type(test)

str

In [93]:
Dict = {}

for i in list(classes):
    page_content = df[df.page == i].parsed.values
    data = ''.join(list(page_content))
    Dict[i] = [data]
    
d = pd.DataFrame.from_dict(Dict,orient='index', columns=['Data'])
d['PageID'] = d.index
sent = list(d.Data.values)

In [114]:
d.head()

,Data,PageID
0,タイトル 設定サブタイトル 設定設定 順序設定 順序設定 順序全て 端末 os 対象オンプレ...,0
1,apple 製品manualapple製品管理ios 端末 業務 アプリ 管理業務 アプリ ...,1
2,android 端末manualandroid設定 モードa. android 端末a-1 ...,2
3,pcmanualpc根本的 セキュリティ対策 リモート 設定貴社 pc 正常 構成 証明図m...,3
4,windows embeddedmanualwindows embedded端末window...,4


In [120]:
d[d['Data'].str.contains('apple')]

,Data,PageID
0,タイトル 設定サブタイトル 設定設定 順序設定 順序設定 順序全て 端末 os 対象オンプレ...,0
1,apple 製品manualapple製品管理ios 端末 業務 アプリ 管理業務 アプリ ...,1
6,利点manual利点20年 歴史製品 機能 ヶ国 お客様 要望 凝縮業務 アプリ サイレント...,6
7,バージョン 機能manualバージョン 機能追加 機能 列挙 バージョンアップ 参考ご存知 ...,7
8,評価 版 設定manual評価 版 設定評価 機会順序 設定管理 コンソール ログインand...,8
...,...,...
429,担当者 アカウント 取得manualapple担当者 アカウント 取得apple 担当者 ア...,429
430,apple 製品manualapple製品管理ios 端末 業務 アプリ 管理業務 アプリ ...,430
431,トークン 入手manualトークン 入手a. appleapple app store 有料...,431
437,プロファイル ルール 詳細 設定manual構成 プロファイル ルール 詳細 設定端末 定常...,437


In [98]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=stop_words_ja, max_df=0.98)
vectors = vectorizer.fit_transform(sent)

In [99]:
vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.98, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ',
                            'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか',
                            'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら',
                            'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [100]:
vectors

<445x18183 sparse matrix of type '<class 'numpy.float64'>'
	with 74376 stored elements in Compressed Sparse Row format>

In [78]:
# idf score 
#idf = vectorizer.idf_
# visulalize idf score
#print(dict(zip(vectorizer.get_feature_names(), idf)))

weights = np.asarray(vectors.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vectorizer.get_feature_names(), 'weight': weights})

In [79]:
TermsWeight = weights_df.sort_values(by='weight', ascending=False).head(50)

In [ ]:
TermsWeight.term

In [123]:
cleaning('iMessageの利用禁止', stop_words_ja)

'利用 禁止 '

In [131]:
MeCab.Tagger().parse('iMessageの利用禁止')

'iMessage\t名詞,固有名詞,組織,*,*,*,*\nの\t助詞,連体化,*,*,*,*,の,ノ,ノ\n利用\t名詞,サ変接続,*,*,*,*,利用,リヨウ,リヨー\n禁止\t名詞,サ変接続,*,*,*,*,禁止,キンシ,キンシ\nEOS\n'

In [103]:
# Read a question from the user
question = input('Please enter a question: \n')

parsed_question = cleaning(question, stop_words_ja)
parsed_question = parsed_question.strip("\n").rstrip()

print("Question: ", parsed_question)

q = vectorizer.transform([parsed_question])

# Rank all the questions using cosine similarity to the input question
rank = cosine_similarity(q, vectorizer.transform(sent))

# Grab the top 5
top = np.argsort(rank, axis=-1).T[-3:]
#print(top)

for item in range(len(top)):
    idx = top[item,0]
    
    # if the dataset file structure "Quesition | Answer" then use following line  
    print("PageID: ",d.iloc[idx,-1],"idx",idx)
    #print("PageID: ",d.iloc[idx,-1],"idx",idx,"=",d.Data.values[idx])
    print("##########")
    #print("PageID:",d.iloc[idx,-1],"idx",idx,"=",d.Data.values[idx][0:300])

Please enter a question: 
iMessageの利用を禁止したい
Question:  利用 禁止
PageID:  364 idx 364
##########
PageID:  296 idx 296
##########
PageID:  330 idx 330
##########


In [ ]:
rank

In [126]:
 rank.argsort()[0][-2]

189

In [128]:
sent[189]

'ランチャー と タグ MobiControl v 14 Manual ランチャー の タグ ランチャー の 登録 の 「 ランチャー と タグ 」 ページ を ご覧 ください 。 詳しく は Copyright © 2010 - 2019 / Translated by Penetrate of Limits Co ., Ltd .( Tokyo ) 無題 ドキュメント © 2019 MobiControl v 14 Manual . WordPress と で 構築 '

In [127]:
def parse(unicode_string):
    tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
    node = tagger.parseToNode(unicode_string)

    words = []
    nouns = []
    verbs = []

    while node:
        pos = node.feature.split(",")[0]
        print(pos + ' ' + node.feature)
        
        word = node.feature.split(",")[-3]
        if pos == "名詞":
            nouns.append(word)
        elif pos == "??":
            verbs.append(word)
        
        words.append(word)
        node = node.next
    parsed_words_dict = {
        "all": words[1:-1], # ??????????????????
        "nouns": nouns,
        "verbs": verbs,
        }
    return parsed_words_dict

In [128]:
parse('iMessageの利用禁止')

BOS/EOS BOS/EOS,*,*,*,*,*,*,*,*
名詞 名詞,固有名詞,組織,*,*,*,*
助詞 助詞,連体化,*,*,*,*,の,ノ,ノ
名詞 名詞,サ変接続,*,*,*,*,利用,リヨウ,リヨー
名詞 名詞,サ変接続,*,*,*,*,禁止,キンシ,キンシ
BOS/EOS BOS/EOS,*,*,*,*,*,*,*,*


{'all': ['*', 'の', '利用', '禁止'], 'nouns': ['*', '利用', '禁止'], 'verbs': []}